In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import data
import plotly.express as px
import pandas   as pd


In [ ]:
path = r"C:\Users\lnae0002\Desktop\autolamella\autolamella\log\3"

df_sample, df_history, df_shift, df_steps  = data.calculate_statistics_dataframe(path)
display(df_shift)

In [ ]:
points = pd.DataFrame(df_shift["shift"].tolist(), columns=["x", "y"])
points["lamella"] = df_shift["lamella"]
fig = px.scatter(points, x="x", y='y', color="lamella")
fig.show()

In [ ]:
display(df_steps)
fig = px.scatter(df_steps, x="step_n", y="timestamp", color="stage", symbol='lamella')
fig.show()



In [ ]:
# calculate difference in timestamp between rows
df_steps['delta'] = df_steps['timestamp'].diff()

display(df_steps)

px.bar(df_steps, x="lamella", y="delta", color="step", facet_col="stage")

In [ ]:
fig = px.bar(df_history, x="stage", y="duration", color="petname", barmode="group")

fig.show()

In [ ]:
# display(df_history)
df_history["duration"].mean()

df_sorted = df_history.groupby(["stage", "petname"]).mean()
df_sorted.drop(columns=["start", "end"], inplace=True)
df_sorted.reset_index(inplace=True)
display(df_sorted)


In [ ]:
points = pd.DataFrame(df_sample["lamella.centre"].tolist(), columns=["x", "y"])
points["petname"] = df_history["petname"].unique()
display(points)
fig = px.scatter(points, x="x", y='y', color="petname")
fig.show()

In [ ]:
df_sample["petname"] = df_history["petname"].unique()
fig = px.scatter_3d(df_sample, x="lamella.x", y='lamella.y', z='lamella.z', color="petname")
fig.show()

### SpaceTOMO

In [ ]:

from PIL import Image
import tifffile as tff
import glob



PATH = "../../../data/spacetomo/imagesTr/"


filenames = glob.glob(PATH + "*.png")

from pprint import pprint

# pprint(filenames)

# convert to tiff, save in separate dir   
for filename in filenames:
    im = Image.open(filename)
    im.save(filename.replace(".png", ".tif"))


### Feature Detection Tools



In [ ]:
%load_ext autoreload
%autoreload 2

import os
import glob
import pandas as pd



path = "../../../data/autolamella-paper/model-development/train/serial-liftout/test/"

filenames = sorted(glob.glob(os.path.join(path, "*.tif*")))


print(df.head())

In [ ]:
df = pd.read_csv(os.path.join(path, "data.csv"))

display(df.head())

filenames = []
for fname in df["filename"].values:
    filenames += glob.glob(os.path.join(path, f"*{fname}*"))

print(len(filenames))

from pprint import pprint
pprint(filenames)


In [ ]:
df = pd.read_csv(os.path.join(path, "data.csv"))


fname = df["filename"].values[0]

df_filt = df[df["filename"] == fname]


display(df)

# drop rows with filename = fname
df.drop(df[df["filename"] == fname].index, inplace=True)

display(df)

#

In [ ]:
import pandas as pd

PATH = "/home/patrick/github/data/autolamella-paper/model-development/train/summary.csv"

df = pd.read_csv(PATH)
# group by metho, dataset, -> count

df_grouped = df.groupby(["method", "dataset"]).count()

display(df_grouped)

# group by method
df_grouped = df.groupby(["method"]).count()

display(df_grouped)

# group by dataset
df_grouped = df.groupby(["dataset"]).count()

display(df_grouped)


### HF Dataset

https://github.com/huggingface/transformers/tree/main/examples/pytorch/semantic-segmentation#note-on-custom-data

In [ ]:
from datasets import Dataset, DatasetDict, Image
import os
import glob


TRAIN_PATH = "/home/patrick/github/data/autolamella-paper/model-development/train/autolamella-waffle/train/"
TEST_PATH = "/home/patrick/github/data/autolamella-paper/model-development/train/autolamella-waffle/test/"


# your images can of course have a different extension
image_paths_train = sorted(glob.glob(os.path.join(TRAIN_PATH, "*.tif*")))
label_paths_train = sorted(glob.glob(os.path.join(TRAIN_PATH, "labels", "*.tif*")))

# same for test
image_paths_test = sorted(glob.glob(os.path.join(TEST_PATH, "*.tif*")))
label_paths_test = sorted(glob.glob(os.path.join(TEST_PATH, "labels", "*.tif*")))

def create_dataset(image_paths, label_paths):
    dataset = Dataset.from_dict({"image": sorted(image_paths),
                                "label": sorted(label_paths)})
    dataset = dataset.cast_column("image", Image())
    dataset = dataset.cast_column("label", Image())

    return dataset

# step 1: create Dataset objects
train_dataset = create_dataset(image_paths_train, label_paths_train)
test_dataset = create_dataset(image_paths_test, label_paths_test)

# step 2: create DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
  }
)

display(dataset)


# step 3: push to hub (assumes you have ran the huggingface-cli login command in a terminal/notebook)
# dataset.push_to_hub("name of repo on the hub")

# optionally, you can push to a private repo on the hub
dataset.push_to_hub("patrickcleeve/autolamella", private=True)

In [ ]:
from datasets import Dataset, DatasetDict, Image, NamedSplit, DatasetInfo, ClassLabel, Features
import os
import glob


WAFFLE_TRAIN_PATH = "/home/patrick/github/data/autolamella-paper/model-development/train/autolamella-waffle/train/"
WAFFLE_TEST_PATH = "/home/patrick/github/data/autolamella-paper/model-development/train/autolamella-waffle/test/"

AUTOLIFTOUT_TRAIN_PATH = "/home/patrick/github/data/autolamella-paper/model-development/train/autoliftout/train/"
AUTOLIFTOUT_TEST_PATH = "/home/patrick/github/data/autolamella-paper/model-development/train/autoliftout/test/"

SERIAL_LIFTOUT_TRAIN_PATH = "/home/patrick/github/data/autolamella-paper/model-development/train/serial-liftout/train/"
SERIAL_LIFTOUT_TEST_PATH = "/home/patrick/github/data/autolamella-paper/model-development/train/serial-liftout/test/"



def create_dataset(image_paths, label_paths, split: str, info=None):

    info = DatasetInfo(
        description="Autolamella Dataset",
        homepage= "https://github.com/DeMarcoLab/autolamella",
        license= "MIT",
        # features= Features({
        #     # "image": Image(),
        #     # "annotation": Image(),
        #     "label": ClassLabel(
        #         num_classes=6,
        #         names=["background", "lamella", "manipulator", "landing_post", "copper_adaptor", "volume_block"]),
        # }),
    )
    # TODO: add features to the dataset

    dataset = Dataset.from_dict({"image": sorted(image_paths),
                                "annotation": sorted(label_paths),
                                # "split": [split] * len(image_paths)
                                # "features": [info.features] * len(image_paths),
                                },
                                split=split,
                                # split=split,
                                # features=info.features, 
                                info=info
                                )
    dataset = dataset.cast_column("image", Image())
    dataset = dataset.cast_column("annotation", Image())


    return dataset

train_paths = [WAFFLE_TRAIN_PATH, AUTOLIFTOUT_TRAIN_PATH, SERIAL_LIFTOUT_TRAIN_PATH]
test_paths = [WAFFLE_TEST_PATH, AUTOLIFTOUT_TEST_PATH, SERIAL_LIFTOUT_TEST_PATH]


autolamella_datasets = {
    "waffle": {"train": {}, "test": {}},
    "liftout": {"train": {}, "test": {}},
    "serial-liftout": {"train": {}, "test": {}},
}

for name, train_path, test_path in zip(autolamella_datasets.keys(), train_paths, test_paths):
    # your images can of course have a different extension
    image_paths_train = sorted(glob.glob(os.path.join(train_path, "*.tif*")))
    label_paths_train = sorted(glob.glob(os.path.join(train_path, "labels", "*.tif*")))

    # same for test
    image_paths_test = sorted(glob.glob(os.path.join(test_path, "*.tif*")))
    label_paths_test = sorted(glob.glob(os.path.join(test_path, "labels", "*.tif*")))

    # step 1: create Dataset objects
    train_dataset = create_dataset(image_paths_train, label_paths_train, "train")
    test_dataset = create_dataset(image_paths_test, label_paths_test, "test")

    autolamella_datasets[name]["train"] = train_dataset
    autolamella_datasets[name]["test"] = test_dataset


# create DatasetDict
waffle_dataset = DatasetDict(autolamella_datasets["waffle"])
liftout_dataset = DatasetDict(autolamella_datasets["liftout"])
serial_liftout_dataset = DatasetDict(autolamella_datasets["serial-liftout"])


# https://huggingface.co/docs/datasets/v2.16.1/en/repository_structure



print(waffle_dataset)
print(liftout_dataset)
print(serial_liftout_dataset)


# optionally, you can push to a private repo on the hub
waffle_dataset.push_to_hub("patrickcleeve/autolamella", config_name="waffle", private=True)
liftout_dataset.push_to_hub("patrickcleeve/autolamella", config_name="liftout",private=True)
serial_liftout_dataset.push_to_hub("patrickcleeve/autolamella", config_name="serial-liftout",private=True)

In [ ]:
# info, features

info = DatasetInfo.from_dict({
    "description": "Autolamella dataset",
    "homepage": "https://github.com/DeMarcoLab/autolamella",
    "license": "MIT",
    "features": Features({
        "label": ClassLabel(num_classes=6, 
                            names=["background", "lamella", "manipulator", "landing_post", "copper_adaptor", "volume_block"])
    }),
})

from pprint import pprint
pprint(info.features)

In [ ]:
from datasets import load_dataset
ds = load_dataset("patrickcleeve/autolamella", name="waffle")

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from fibsem.segmentation.utils import decode_segmap_v2

# random data
idx = random.randint(0, len(ds["train"]))
image = np.asarray(ds["train"][idx]["image"])
mask = np.asarray(ds["train"][idx]["annotation"])

# metadata
split = ds["train"].split
config_name = ds["train"].config_name

plt.title(f"{config_name}-{split}-{idx:02d}")
plt.imshow(image, cmap="gray", alpha=0.7)
plt.imshow(decode_segmap_v2(mask), alpha=0.3)
plt.axis("off")
plt.show()


In [ ]:
images = [np.asarray(img) for img in ds["train"]["image"]]
masks = [np.asarray(mask) for mask in ds["train"]["annotation"]]


### Validate Dataset


In [ ]:
%load_ext autoreload
%autoreload 2

import os
import glob
import pandas as pd

PATH = "/home/patrick/github/data/autolamella-paper/model-development/train/"

methods = ["autolamella-waffle", "autoliftout", "serial-liftout"]
splits = ["train", "test"]

df = pd.read_csv(os.path.join(PATH, "summary2.csv"))

display(df.head())

print(len(df))

# loop over methods, splits, check that each file exists

to_drop = []
for method in methods:
    for split in splits:
        df_filt = df[(df["method"] == method) & (df["dataset"] == split)]
        print(f"Method: {method}, Split: {split}, Num files: {len(df_filt)}")
        for fname in df_filt["filename"].values:
            if not os.path.exists(os.path.join(PATH, method, split, fname)):
                print(f"File {fname} does not exist")

                to_drop.append((fname, split))

# group by method, dataset, -> count

# drop rows with filename = fname

# show rows in to_drop

# drop rows in to_drop
for fname, split in to_drop:
    df.drop(df[(df["filename"] == fname) & (df["dataset"] == split)].index, inplace=True)

    

df_grouped = df.groupby(["method", "dataset"]).count()
display(df_grouped)
print(len(df))

# save to csv
# df.to_csv(os.path.join(PATH, "summary2.csv"), index=False)


In [ ]:
from datasets import load_dataset

ds = load_dataset("patrickcleeve/autolamella", name="waffle")
print(ds)
ds = load_dataset("patrickcleeve/autolamella", name="liftout")
print(ds)
ds = load_dataset("patrickcleeve/autolamella", name="serial-liftout")
print(ds)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from fibsem.segmentation.utils import decode_segmap_v2
import tifffile as tff
import random


df = pd.read_csv(os.path.join(PATH, "summary.csv"))

split = "train"
method = "waffle"

df_filt = df[(df["method"] == method) & (df["split"] == split)]
print(f"Method: {method}, Split: {split}, Num files: {len(df_filt)}")


idx = random.randint(0, len(df_filt))
fname = df_filt["filename"].values[idx]
print(idx, fname)

ds = load_dataset("patrickcleeve/autolamella", name=method, split=split)

image = tff.imread(os.path.join(PATH, method, split, fname))
mask = tff.imread(os.path.join(PATH, method, split, "labels", fname))

image = np.asarray(ds[idx]["image"])
mask = np.asarray(ds[idx]["annotation"])

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

fig.suptitle(f"{method}-{split}-{idx:02d}")
ax[0].imshow(image, cmap="gray", alpha=0.7)
ax[0].imshow(decode_segmap_v2(mask), alpha=0.3)
ax[0].axis("off")
ax[1].imshow(image, cmap="gray", alpha=0.7)
ax[1].imshow(decode_segmap_v2(mask), alpha=0.3)
ax[1].axis("off")
plt.show()

print(df_filt.values[idx])



## Combine multiple datasets



In [ ]:
from datasets import load_dataset, concatenate_datasets
waffle_train_ds = load_dataset("patrickcleeve/autolamella", name="waffle", split="train")
liftout_train_ds = load_dataset("patrickcleeve/autolamella", name="liftout", split="train")
serial_liftout_train_ds = load_dataset("patrickcleeve/autolamella", name="serial-liftout", split="train")

In [ ]:
from datasets import load_dataset, concatenate_datasets

# load invidual datasets
waffle_train_ds = load_dataset("patrickcleeve/autolamella", name="waffle", split="train")
liftout_train_ds = load_dataset("patrickcleeve/autolamella", name="liftout", split="train")
serial_liftout_train_ds = load_dataset("patrickcleeve/autolamella", name="serial-liftout", split="train")

# concatenate datasets (e.g. mega model)
train_ds = concatenate_datasets([waffle_train_ds, liftout_train_ds, serial_liftout_train_ds])

print(train_ds)


### Volume Block Corners




In [ ]:
%load_ext autoreload
%autoreload 2

from datasets import load_dataset, concatenate_datasets
import matplotlib.pyplot as plt 
import numpy as np
from fibsem.segmentation.utils import decode_segmap_v2

ds = load_dataset("patrickcleeve/autolamella", name="serial-liftout", split="train")


image = np.asarray(ds[4]["image"])
mask = np.asarray(ds[4]["annotation"])


from fibsem.detection.detection import VolumeBlockTopLeftCorner, VolumeBlockTopRightCorner, VolumeBlockBottomLeftCorner, VolumeBlockBottomRightCorner

top_left_corner = VolumeBlockTopLeftCorner().detect(None, mask)
top_right_corner = VolumeBlockTopRightCorner().detect(None, mask)
bottom_left_corner = VolumeBlockBottomLeftCorner().detect(None, mask)
bottom_right_corner = VolumeBlockBottomRightCorner().detect(None, mask)


plt.imshow(image, cmap="gray", alpha=0.7)
plt.imshow(decode_segmap_v2(mask), alpha=0.3)
plt.scatter(top_left_corner[0], top_left_corner[1], color="magenta")
plt.scatter(top_right_corner[0], top_right_corner[1], color="magenta")
plt.scatter(bottom_left_corner[0], bottom_left_corner[1], color="magenta")
plt.scatter(bottom_right_corner[0], bottom_right_corner[1], color="magenta")
plt.axis("off")
plt.show()



In [ ]:

from fibsem import utils
from fibsem.patterning import _get_milling_stages
from fibsem.ui.utils import _draw_milling_stages_on_image 
from fibsem.structures import Point
from copy import deepcopy
import numpy as np


microscope, settings = utils.setup_session()

protocol = utils.load_protocol("protocol.yaml")
settings.protocol = protocol

# get evenly spaced points
width = 100e-6
n_patterns = 4
pos_x = np.linspace(-width/2, width/2, n_patterns)
points = [Point(x, 0) for x in pos_x]

# acquire image for visualiation
settings.image.hfw = 150e-6
image = microscope.acquire_image(settings.image)

block_stages = []
for i, pt in enumerate(points):

    # get milling stage
    stage = _get_milling_stages("copper-block-removal", deepcopy(settings.protocol), deepcopy(pt))[0]
    stage.name = f"Copper-Block-Removal-{i:02d}"
    block_stages.append(deepcopy(stage))

# get top removal stage
top_stages= _get_milling_stages("copper-block-removal-top", settings.protocol, Point(0, 10e-6))

stages = top_stages + block_stages

# draw stages on image
fig = _draw_milling_stages_on_image(image, stages)





## Apply Microscope Configuration


In [ ]:
%load_ext autoreload
%autoreload 2


from fibsem import utils
from pprint import pprint

microscope, settings = utils.setup_session()

# apply configuration
utils.apply_configuration(microscope, settings)



In [ ]:
from fibsem.structures import BeamType




# # beam system settings
# electron_system_settings = microscope.get_beam_system_settings(BeamType.ELECTRON)
# ion_system_settings = microscope.get_beam_system_settings(BeamType.ION)

# pprint(electron_system_settings.__to_dict__())
# pprint(ion_system_settings.__to_dict__())

# electron_settings = microscope.get_beam_settings(BeamType.ELECTRON)
# ion_settings = microscope.get_beam_settings(BeamType.ION)

# pprint(electron_settings.__to_dict__())
# pprint(ion_settings.__to_dict__())


# # detector settings
# electron_detector_settings = microscope.get_detector_settings(BeamType.ELECTRON)
# ion_detector_settings = microscope.get_detector_settings(BeamType.ION)

# pprint(electron_detector_settings.__to_dict__())
# pprint(ion_detector_settings.__to_dict__())

# check that microscope and settings are the same
assert microscope.hardware_settings == settings.hardware
assert microscope.stage_settings == settings.system.stage

## Microscope Fingerprint

In [ ]:

microscope, settings = utils.setup_session()

electron_system_settings = microscope.get_beam_system_settings(BeamType.ELECTRON)
ion_system_settings = microscope.get_beam_system_settings(BeamType.ION)
imaging_settings = microscope.get_imaging_settings(BeamType.ELECTRON)


pprint(electron_system_settings.__to_dict__())
pprint(ion_system_settings.__to_dict__())

pprint(imaging_settings.__to_dict__())




In [ ]:
pprint(microscope.system.__to_dict__())



# check beams are on

# check chamber is pumped




In [1]:
# validate system

from fibsem import utils
from fibsem.structures import BeamType
from fibsem.microscope import FibsemMicroscope
import logging

microscope, settings = utils.setup_session()

print(microscope.get("on", BeamType.ELECTRON))
print(microscope.get("on", BeamType.ION))

# microscope.set("on", True, BeamType.ELECTRON)
# microscope.set("on", True, BeamType.ION)


# get the system is pumped
print("Chamber State: ", microscope.get("chamber_state"))
print("Chamber Pressure: ", microscope.get("chamber_pressure"))

# set vent
microscope.set("vent", True)

print("Chamber State: ", microscope.get("chamber_state"))
print("Chamber Pressure: ", microscope.get("chamber_pressure"))

from fibsem.validation import validate_microscope

warnings = validate_microscope(microscope)

print(warnings)


Default configuration: default-configuration
Default configuration path: /home/patrick/github/fibsem/fibsem/config/microscope-configuration.yaml
2024-01-17 00:40:23,601 — root — INFO — connect_to_microscope:5231 — Connected to Demo Microscope
2024-01-17 00:40:23,601 — root — INFO — connect_to_microscope:5232 — Microscope client connected to model Demo with serial number 123456 and software version 0.1
2024-01-17 00:40:23,602 — root — INFO — reset_beam_shifts:5339 — Resetting beam shifts
2024-01-17 00:40:23,602 — root — INFO — setup_session:221 — Finished setup for session: demo_2024-01-17-12-40-23AM
False
False
Chamber State:  Pumped
Chamber Pressure:  1e-06
2024-01-17 00:40:23,602 — root — INFO — set:5860 — Setting vent to True (None)
Chamber State:  Vented
Chamber Pressure:  100000.0


/home/patrick/miniconda3/envs/fibsem/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-01-17 00:40:25,479 — root — WARNING — get:5856 — Unknown key: stage_homed (None)
2024-01-17 00:40:25,480 — root — WARNING — get:5856 — Unknown key: stage_linked (None)
2024-01-17 00:40:25,480 — root — WARNING — get:5856 — Unknown key: manipulator_state (None)
2024-01-17 00:40:25,481 — root — WARNING — validate_microscope:335 — Microscope Validation Warnings: ['Electron beam is off', 'Ion beam is off', 'Chamber is not pumped', 'Stage is not homed', 'Stage is not linked', 'Needle is not retracted']
['Electron beam is off', 'Ion beam is off', 'Chamber is not pumped', 'Stage is not homed', 'Stage is not linked', 'Needle is not retracted']
